In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
IMG_SIZE = 64
BATCH_SIZE = 64
DATA_DIR = "asl_alphabet_train"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Save label map
with open("label_map.json", "w") as f:
    json.dump(train_generator.class_indices, f)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(29, activation='softmax')
])

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 0.2718 - loss: 2.5525

In [ ]:
model.save("asl_cnn_model.h5")
# Load model and label map
model = load_model("asl_cnn_model.h5")
with open("label_map.json", "r") as f:
    class_indices = json.load(f)
    labels = {v: k for k, v in class_indices.items()}

# Load image and preprocess
img_path = "asl_alphabet_test/C_test.jpg"  # change to your image
img = cv2.imread(img_path)
img = cv2.resize(img, (64, 64))
img = img.astype("float") / 255.0
img = np.expand_dims(img, axis=0)

# Predict
pred = model.predict(img)[0]
pred_label = labels[np.argmax(pred)]
confidence = np.max(pred)

print(f"Prediction: {pred_label.upper()}, Confidence: {confidence*100:.2f}%")

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# --- 2. Load your trained model ---
model = load_model("asl_cnn_model.h5")  # ✅ This should already exist from earlier

# --- 3. Set the image size and class labels ---
IMG_SIZE = 64  # or whatever your model was trained with
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
    'U', 'V', 'W', 'X', 'Y', 'Z',
    'DELETE', 'NOTHING', 'SPACE']  # all 29 classes

# --- 4. (Optional) test with static image ---
# Code to test a single image, if you want to try before webcam

# --- 5. ADD THIS BLOCK AT THE END — Real-Time Webcam Detection ---
cap = cv2.VideoCapture(0)
print("Starting real-time ASL detection... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for prediction
    roi = cv2.resize(frame, (64, 64))
    roi = roi.astype("float32") / 255.0
    roi = np.expand_dims(roi, axis=0)

    # Predict the sign
    prediction = model.predict(roi)
    predicted_class = np.argmax(prediction)
    print("Predicted index:", predicted_class)
    print("Total labels:", len(class_names))
    label = class_names[predicted_class]

    # Show result on video
    cv2.putText(frame, f'Predicted: {label}', (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    cv2.imshow("ASL Real-Time Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()